# Mensen

## Inhaltsverzeichnis
- [Vorbereitungen](#Vorbereitungen)
- [Pommes-Frites Angebot](#Pommes-Frites-Angebot)
- [Preisniveau der Gerichte](#Preisniveau-der-Gerichte)

## Vorbereitungen

Import der benötigten Module:

In [ ]:
import tarfile
from datetime import datetime
from typing import Dict, List, Any
from bs4 import BeautifulSoup

import pandas as pd
import plotly.express as px

Laden der Metadaten in den Arbeitsspeicher:

In [ ]:
mensen_info: pd.DataFrame = pd.read_csv('mensen_info.csv', encoding='utf-8')
mensen_info.sample(n=5, random_state=1)

Laden aller HTML-Dateien, parsen mit BeautifulSoup und speichern in einem DataFrame:

In [ ]:
# temporäre Liste anlegen
data: List[Dict[str, Any]] = []

# über alle Dateien im Tarball iterieren
with tarfile.open('mensen_html.tar.gz', 'r:gz', encoding='utf-8') as tar:
    for member in tar.getmembers():
        with tar.extractfile(member) as file:
            # Mensa ID und Datum extrahieren
            mensa_id, year, month, day = map(int, member.name[:-5].split('-'))
            date = datetime(year=year, month=month, day=day, hour=12)

            # HTML parsen
            html = BeautifulSoup(file, 'html.parser')

            # Container finden und angebotsfreie Tage überspringen
            # container = html.find('div', {'class': 'splGroupWrapper'})

            #if container is None:
            #    h2 = html.find('h2', {'class': {'mt-3'}})
            #    if h2 is not None and h2.text == 'Zum gewählten Datum werden in dieser Einrichtung keine Essen angeboten.':
            #        continue

            #if container is None:
            #    continue

            # Gerichte finden
            for meal in html.find_all('div', {'class': 'rowMeal'}):
                # Daten auslesen
                name = meal.find('div', {'class': 'mealText'}).text
                name = name.strip()

                prices = meal.find('div', {'class': 'mealPreise'}).text
                prices = ','.join(map(str.strip, map(lambda x: x.replace(',', '.'), prices[:-2].split('/'))))

                categories = meal.find_all('img', {'class': 'splIconMeal'})
                categories = ','.join([i['title'] for i in categories])

                allergens = meal.find('div', {'class': 'allergene'})
                if allergens is None:
                    allergens = []
                else:
                    allergens = ','.join(map(str.strip, allergens.text.split(':')[1].split(',')))

                # zu temporärer Liste hinzufügen
                data.append({
                    'id': mensa_id,
                    'date': date,
                    'name': name,
                    'prices': prices,
                    'categories': categories,
                    'allergens': allergens,
                })

# in DataFrame konvertieren und Liste löschen
df = pd.DataFrame(data)
del data

# Auszüge aus DataFrame ausgeben
df.sample(n=10, random_state=1)

## Pommes-Frites Angebot
**In welcher Stadt lohnt sich das Studium, wenn man eigentlich nur an Pommes Frites interessiert ist?**

Zuerst wird ein neues DataFrame erstellt, das nur die Stadt, den Wochentag und die Information, ob es Pommes gibt, enthält.

In [ ]:
df_fries = pd.DataFrame()

df_fries['city'] = pd.merge(df, mensen_info, how='left', on='id')['city']
# df_fries['name'] = df['name']
df_fries['fries'] = df['name'].map(lambda x: 'pommes' in x.lower())
df_fries['day_of_week'] = df['date'].map(datetime.isoweekday)

df_fries.sample(n=10, random_state=2)

Anschließend wird nach der Stadt und dem Wochentag gruppiert und die Summe gebildet. (`True` zählt als Summand $1$, `False` als $0$.)

In [ ]:
fries_city_dow = df_fries.groupby(['city', 'day_of_week']).sum()
fries_city_dow.head(7)

Danach wird die entstandene Tabelle in ein DataFrame konvertiert, das reihenweise die Wochentage und spaltenweise die verschiedenen Städte repräsentiert. Im Zuge dessen werden `NaN` Werte mit $0$ ersetzt, die Namen der Spalten zurückgesetzt und die Zahlwerte der Wochentage gegen Namen getauscht.

In [ ]:
fries = fries_city_dow.swaplevel(i=0, j=1).unstack(fill_value=0).reset_index()
fries.columns = [col[1 if col[0] == 'fries' else 0] for col in fries.columns.values]
fries['day_of_week'] = fries['day_of_week'].replace({
    1: 'Mo',
    2: 'Di',
    3: 'Mi',
    4: 'Do',
    5: 'Fr',
    6: 'Sa',
    7: 'So',
})

fries

In [ ]:
fig = px.bar(fries, x='day_of_week', y=fries.columns[1:], barmode='group',
             title='Anzahl der Pommes-Frites Gerichte nach Stadt',
             labels={'day_of_week': 'Wochentag', 'variable': 'Stadt'})
fig.update_layout(yaxis_title='Anzahl')

fig

Gera und Eisenach sind also zu meiden, während Jena und Ilmenau empfehlenswert sind.

## Preisniveau der Gerichte
**Wie entwicklen sich die Preise in verschiedenen Städten?**

An dieser Stelle sollen die Preise für Studenten betrachtet werden. Zunächst wird also die Spalte `prices` von anderen Preisen bereinigt und in `float` konvertiert. Ebenfalls werden alle Gerichte anhand des Datums der Kalenderwoche zugeordnet.

In [ ]:
df_std = df.copy()

df_std['price'] = df_std['prices'].str.split(',').str[0]
df_std['price'] = pd.to_numeric(df_std['price'])

df_std['week'] = df_std['date'].map(lambda x: f'{x.year}-{x.week}')

df_std.sample(n=5, random_state=1)

Anschließend wird nach der entsprechenden Mensa und der Woche gruppiert und die Indizes vertauscht, um den Durchschnittspreis jeder Woche jeder Mensa berechnen zu können.

In [ ]:
s_grouped = df_std.groupby(['id', 'week']).mean().swaplevel(i=0, j=1).reset_index()

Eine Verknüpfung mit den Daten in `mensen_info` erlaubt, zu jeder Mensa die Stadt zuzuordnen.

In [ ]:
df_merged = pd.merge(
    s_grouped, mensen_info,
    how='left', on='id'
)
df_merged.sample(n=5, random_state=1)

Zuletzt erfolgt die Darstellung.

In [ ]:
# ToDo Darstellung überarbeiten
px.line(df_merged, x='week', y='price', color='city')

ToDo: **Zusammenfassung**